## Hands-on session 2.2 - Variable-like modelbound data

This short sessions takes a break from plasma physics as we play around with the [predator-prey model](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations). 

Demonstrated concepts:

- 0D simulations with ReMKiT1D
- Variable-like modelbound data
- Automatic generation of modelbound data using nodes
- Setting time target integration
- Using RK4 and setting term groups
- Building a simple composite integrator

In [ ]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm

import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt


### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for writing the files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/day_2_2/"
rk.setHDF5Path(hdf5Filepath) 

### Grid initialization

We initialize a trivial 0D grid

In [ ]:
rk.grid = Grid(np.zeros(1))

### The equations

The predator-prey system can be written as follows:

$$ \frac{dx}{dt} = (\alpha - \beta y)x $$
$$ \frac{dy}{dt} = (\delta x - \gamma)y $$

where $\alpha$ and $\beta$ are the prey growth and death factors, and $\delta$ and $\gamma$ are the predator growth and death factors. $x$ and $y$ are prey and predator numbers in arbitrary units.

### Variables

We just need the implicit predator and prey variables, and can also set the 4 constants while we're at it. Let's also set two more copies of x and y so we can build the model in two ways.

**NOTE**: The `x` name is reserved for the grid, so we do not use it for variables. Use x1,y1,x2,y2.

Initialize both of the x variables to 10, and both the y variables to 2 (remember, they should be numpy arrays with the length of the grid!)

Do not forget to add the `time` derived scalar variable. We will explicitly need it in this session.

In [ ]:
# [YOUR CODE HERE]

alpha = 1.1
beta = 0.4
delta = 0.1
gamma = 0.4

### The model - general

We can build the model in two ways because of its structure. The general way comes first, where we introduce modelbound data by hand.

Let's first add the derivations we need to the wrapper. These should evaluate to the bracketed values in the equations.

Use the `addCustomDerivation` method on the wrapper to add the two derivations, and add the derivations using the `treeDerivation` function and using `Node` where necessary. Use the first two variables here (x1,y1)!

In [ ]:
#[YOUR CODE HERE]

Let's then build a model for the predator-prey equations and add create the modelbound data that goes with it

Use `sc.VarlikeModelboundData` and the corresponding `add.Variable` methods. Do not forget to call `setModelboundData` (you'll need to call `dict()` on the modelbound data object before passing it to the wrapper)

In [ ]:
predPreyModel = sc.CustomModel("predatorPrey")

# [YOUR CODE HERE]


Let's now add the two simple terms by hand

Take the modelbound variables you added and use them as `reqMBRowVars`

In [ ]:
xTerm = sc.GeneralMatrixTerm("x1",
                             varData=sc.VarData(reqMBRowVars=[""]),
                             stencilData=sc.diagonalStencil())

predPreyModel.addTerm("xTerm",xTerm)

yTerm = sc.GeneralMatrixTerm("y1",
                             varData=sc.VarData(reqMBRowVars=[""]),
                             stencilData=sc.diagonalStencil(),
                             implicitGroups=[2]) # See note below

predPreyModel.addTerm("yTerm",yTerm)

rk.addModel(predPreyModel)

**NOTE**: 

Term groups: We can have finer control over time integration by setting term groups. This is particularly important when using the explicit RK integrators, as they will skip any mixed term groups - groups of terms with different evolved variables. A way to avoid worrying about this is not to have models which evolve more than 1 variable.

### The model - a faster way

As noted above, because the structure of our equations looks like rowVariable * implicitVariable, we can use the `addNodeMatrixTermModel` function from `common_models` to automatically add the modelbound variables and the terms

The `termDefs` below are missing the `Node` objects they should use for the derivation. These should be the same objects as the ones you used above. Make sure they use the correct variables (x2,y2)!

In [ ]:
cm.addNodeMatrixTermModel(rk,
                          modelTag="predatorPreyQuickerX",
                          evolvedVar="x2",
                          termDefs=[(,"x2")]) #[YOUR CODE HERE]

cm.addNodeMatrixTermModel(rk,
                          modelTag="predatorPreyQuickerY",
                          evolvedVar="y2",
                          termDefs=[(,"y2")]) #[YOUR CODE HERE]

### Setting up the time integration options

The predator-prey model is a good place to showcase the different behaviour of the two time integrators available. The RK integrator goes up to fourth order, while the Backwards Euler integrator is first order. 

We can use ReMKiT1D's composite integrator feature to add multiple integrators and split integration so that different models are evolved using different integrators. 

Let's first add the integrators:

In [ ]:
integratorRK = sc.rkIntegrator(4) # Play around with the order value for RK to see how the solution is affected
integratorBDE = sc.picardBDEIntegrator(nonlinTol=1e-14,absTol=10.0, convergenceVars=["x2","y2"])

rk.addIntegrator("RK", integratorRK)
rk.addIntegrator("BE", integratorBDE)

Unlike in previous sessions, we have split the terms of one of our models into two groups. So we need to tell the composite integrator to expect two groups.

In [ ]:
rk.setIntegratorGlobalData(initialTimestep=0.01,numImplicitGroups=2) # We tell the integrators to expect 2 implicit groups

Let's add the RK step. By default, all steps step for the full duration of the time step, and since we aren't doing operator splitting, we'll keep this behaviour.

However, the `predatorPrey` model has two implict term groups, so we need to tell the integration step to update and evaluate both.

In [ ]:
rkStep = sc.IntegrationStep("RK",defaultEvaluateGroups=[1,2],defaultUpdateGroups=[1,2]) # Tell the RK step to update and evaluate terms in both groups

rkStep.addModel("predatorPrey")

rk.addIntegrationStep("RK1", rkStep.dict())

The Backwards Euler integrator should handle the other two models

In [ ]:
bdeStep = sc.IntegrationStep("BE") # The other two models don't have their terms split into groups

bdeStep.addModel("predatorPreyQuickerX")
bdeStep.addModel("predatorPreyQuickerY")

rk.addIntegrationStep("BE1", bdeStep.dict())


Let's run the predator-prey model up to t=50. We can move from using a fixed number of timesteps to asking the code to run until some minimum time has elapsed. We can also set the output option to write files with a fixed minimum time interval between them.

In [ ]:
rk.setTimeTargetTimestepping(50.0) # Run until 50 time units have elapsed
rk.setMinimumIntervalOutput(0.1) # Output every 0.1 time unit (might not have exactly 500 files because of rounding)

### Create config 

Remember to run ReMKiT1D with a single MPI process for this session.

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=471
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames)
loadedData


### Comparing the integrators

As you can see below, with the same step length, the higher order RK integrator is not as dissipative as the implicit integrator, as expected.

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluidMultiComparison(["x1","y1","x2","y2"],fixedPosition=True)

In [ ]:
fig,ax = plt.subplots(1,2,sharey="row")

ax[0].plot(loadedData["x1"],loadedData["y1"])

ax[0].title.set_text("RK4")

ax[0].set_xlabel("x")
ax[0].set_ylabel("y")

ax[1].plot(loadedData["x2"],loadedData["y2"])

ax[1].title.set_text("First order Backwards Euler")

ax[1].set_xlabel("x")
